In [1]:
from filamentlib.VField import VField
import numpy as np
import timeit

# BiotSavart Algorithm Tests

Generate points to run through the different algorithms

In [2]:
# Define the filament shape
circle = lambda t : [np.cos(t), np.sin(t), t * 0]
dCircle = lambda t : [-np.sin(t), np.cos(t), t * 0]
circlePoints = np.linspace(0,2*np.pi,1000)

# Calculate points on the circle to use
s = np.array(circle(circlePoints))
ds = np.array(dCircle(circlePoints))

# Find points to calculate the Velocity Field at around the circle
# In this case, it will be points along the x-axis through the circle at y=0 and z=0
domain = np.linspace(-1.5, 1.5, 1000)
posFunc = lambda t: [ t, t*0, t*0 ]
pos = np.array( posFunc( domain ) )

print(pos)

[[-1.5        -1.496997   -1.49399399 ...  1.49399399  1.496997
   1.5       ]
 [-0.         -0.         -0.         ...  0.          0.
   0.        ]
 [-0.         -0.         -0.         ...  0.          0.
   0.        ]]


## Algorithm One
This is the first algorithm that I came up with. It works fine, but it's so slow.

In [ ]:
class AlgorithmOne():
    @staticmethod
    def BiotSavartPoint(curve, curveTangent, curvePoint):
        s = range(0,len(curve[0]))
        dv = []
        for c in s:
            num = np.cross( curveTangent[:,c], curvePoint - curve[:,c] )
            den = np.linalg.norm(curvePoint - curve[:,c]) ** 3
            dv.append( np.divide(num,den) )

        dv = np.array(dv)

        v = np.trapz([dv[:,0],dv[:,1],dv[:,2]], s)
        return v
    
    @staticmethod
    def BiotSavartPoints(curve, curveTangent, curvePoints):
        biotSavartPoints = []

        for i in range( len(curvePoints[0,:]) ):
            curvePoint = curvePoints[:,i]
            biotSavartPoints.append( AlgorithmOne.BiotSavartPoint( curve, curveTangent, curvePoint ) )
            
        return np.array( biotSavartPoints )

In [ ]:
startTime = timeit.default_timer()

bsPtsAlg1 = AlgorithmOne.BiotSavartPoints( s, ds, pos )

stopTime = timeit.default_timer()

print(f'Code Time: {stopTime - startTime}')

## Algorithm Two


In [26]:
class AlgorithmTwo():
    @staticmethod
    def BiotSavart( curve: np.array, curveTangent: np.array, fieldPoints: np.array ):
        pointFieldStrength = np.zeros( (len(curve[0,:]), len( fieldPoints[:,0] ), len( fieldPoints[0,:] ) ) )

        s = range( len(curve[0,:]) )

        for i in s:
            pointDistances =  fieldPoints - curve[:,i].reshape(-1,1)

            pointNorms = np.linalg.norm(pointDistances, axis=0)
            pointNormsCubed = np.power( pointNorms, 3)

            crossProduct = ( np.cross(curveTangent[:,i], pointDistances.T ) ).T
            pointFieldStrength[i,:,:] = crossProduct / pointNormsCubed

        v = np.trapz( pointFieldStrength, s , axis=0)
        return v

In [27]:
bsPtsAlg2 = AlgorithmTwo.BiotSavart( s, ds, pos )

In [ ]:
startTime = timeit.default_timer()

bsPtsAlg2 = AlgorithmTwo.BiotSavart( s, ds, pos )

stopTime = timeit.default_timer()

print(f'Code Time: {stopTime - startTime}')

## Summary

The second algorithm is so much faster and the values are very similar. Interestingly, they aren't the same, so there probably should be some error graphs to go along with the calculation. Even if the other algorithm is more accurate, the time difference is so dramatic that it still probably wouldn't be worthwhile to stick with the first algoritm.

In the future, I think if we could get rid of the forloop alltogether, that would be the most ideal algorithm. The idea I currently have is to use Tensors to accomplish this.